In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from collections import Counter

### Load Data

In [3]:
df = pd.read_csv("15min_dateset.csv")
df.drop('Unnamed: 0', inplace = True, axis = 1)
print(len(df['EXP'].unique()))
df.head()

32


,Date,Type,from,EXP,jul_date,hour_angle,max_wind_speed,avg_wind_speed,R1,R2,...,HYBL_one,HYBL_two,HYBL_three,HYBL_four,HYBL_five,diff_temp,geo_cbl,veg_sfc,best_4_layer,ws_700
0,2017-01-07 06:00:00,ANDE,MESONET,1/7/17,7,-1.570795,2.332608,1.295893,2.630564,2.322274,...,0.629230,0.903721,3.610055,5.011670,6.054984,0.739,-5000.0,40.293,23.757,16.787335
1,2017-01-07 06:15:00,ANDE,MESONET,1/7/17,7,-1.505345,1.555072,0.647947,2.055383,1.813005,...,0.748575,1.493288,3.643217,5.232982,6.394255,1.050,-5000.0,40.293,23.959,16.195689
2,2017-01-07 06:30:00,ANDE,MESONET,1/7/17,7,-1.439895,2.332608,1.360688,1.792587,1.671478,...,1.154196,1.645343,3.346854,5.054551,6.395084,1.211,-5000.0,40.293,24.118,15.636587
3,2017-01-07 06:45:00,ANDE,MESONET,1/7/17,7,-1.374446,1.555072,0.453563,1.442889,1.352497,...,2.183654,2.266223,3.550607,5.363623,6.869857,0.943,-5000.0,40.293,24.220,16.767452
4,2017-01-07 07:00:00,ANDE,MESONET,1/7/17,7,-1.308996,2.526992,1.360688,2.283417,1.677404,...,2.407290,2.633109,3.692489,5.309871,6.783159,0.751,-5000.0,40.293,24.149,17.249375


### Functions Used

In [4]:
def model_info1(x, y, test_date, data_tested):
    cm = confusion_matrix(x,y)
    acc = accuracy_score(x,y)
    cm = confusion_matrix(x,y)
    if len(cm) > 1:
        hit = cm[1][1]
        false_alarms = cm[0][1]
        misses = cm[1][0]
        no_events = cm[0][0]
        csi = hit / (false_alarms + misses + hit)
        hit_rate = hit / (hit + misses)
        false_alarm_rate = false_alarms / (hit + false_alarms)
    else:
        hit = 0
        false_alarms = 0
        misses = 0
        no_events = cm[0][0]
        csi = 0
        hit_rate =0
        false_alarm_rate = 0
    data = [{"Accuracy":acc,"Hits":hit, "False Alarms":false_alarms, 'Misses':misses,'No Events': no_events, 'CSI':csi, 'Hit Rate':hit_rate, 'False Alarm Rate':false_alarm_rate}]
    df = pd.DataFrame(data)
    df['test_date'] = test_date
    df['data_tested'] = data_tested
    if len(Counter(x)) == 1:
        l30 = Counter(x)[0]
        df['greter_than_30_knots'] = '0'
        df['less_than_30_knots'] = l30
    else:
        g30 = Counter(x)[1]
        l30 = Counter(x)[0]
        df['greter_than_30_knots'] = g30
        df['less_than_30_knots'] = l30
    return (df)

def model_info2(x, y,test_date,data_tested):
    mae = mean_absolute_error(x,y)
    mse = mean_squared_error(x,y)
    r2 = r2_score(x,y)
    residual_0 = x - y
    res_0 = np.array(residual_0)
    res_mean = round(res_0.mean(),4)
    data = [{"MAE":mae ,"MSE":mse, 'Residual Mean':res_mean}]
    df = pd.DataFrame(data)
    df['test_date'] = test_date
    df['data_tested'] = data_tested
    return (df)

In [13]:
df2 = df[['Date','Type', 'from', 'EXP',
       'max_wind_speed', 'R1', 'R2',
       'Pressure_reduced_to_MSL_.Pa..0.MSL',
       'Derived_radar_reflectivity_.dB..1.HYBL',
       'u.component_of_wind_.m.s..85000.ISBL',
       'v.component_of_wind_.m.s..85000.ISBL', 'wind_speed_85000',
       'wind_shear_85000', 'Wind_speed_.gust._.m.s..0.SFC', 'HYBL_one',
       'HYBL_two', 'HYBL_three', 'HYBL_four', 'HYBL_five', 'diff_temp',
       'geo_cbl', 'veg_sfc', 'best_4_layer', 'ws_700']]
le = LabelEncoder()
df2['from'] = le.fit_transform(df2['from'])
df2.head()

,Date,Type,from,EXP,max_wind_speed,R1,R2,Pressure_reduced_to_MSL_.Pa..0.MSL,Derived_radar_reflectivity_.dB..1.HYBL,u.component_of_wind_.m.s..85000.ISBL,...,HYBL_one,HYBL_two,HYBL_three,HYBL_four,HYBL_five,diff_temp,geo_cbl,veg_sfc,best_4_layer,ws_700
0,2017-01-07 06:00:00,ANDE,1,1/7/17,2.332608,2.630564,2.322274,102879.7,0.0,5.447,...,0.629230,0.903721,3.610055,5.011670,6.054984,0.739,-5000.0,40.293,23.757,16.787335
1,2017-01-07 06:15:00,ANDE,1,1/7/17,1.555072,2.055383,1.813005,102951.0,0.0,5.629,...,0.748575,1.493288,3.643217,5.232982,6.394255,1.050,-5000.0,40.293,23.959,16.195689
2,2017-01-07 06:30:00,ANDE,1,1/7/17,2.332608,1.792587,1.671478,102890.6,0.0,5.653,...,1.154196,1.645343,3.346854,5.054551,6.395084,1.211,-5000.0,40.293,24.118,15.636587
3,2017-01-07 06:45:00,ANDE,1,1/7/17,1.555072,1.442889,1.352497,102931.0,0.0,5.692,...,2.183654,2.266223,3.550607,5.363623,6.869857,0.943,-5000.0,40.293,24.220,16.767452
4,2017-01-07 07:00:00,ANDE,1,1/7/17,2.526992,2.283417,1.677404,102882.5,0.0,5.226,...,2.407290,2.633109,3.692489,5.309871,6.783159,0.751,-5000.0,40.293,24.149,17.249375


### Grid search for best parameters

In [14]:
#X = df2.drop(['max_wind_speed', 'Type', 'EXP'], 1)
#Y = df2.max_wind_speed.values

In [15]:
#from sklearn.model_selection import GridSearchCV
#pram = {'subsample': [0.6, 0.8, 1.0],
        #'colsample_bytree': [0.6, 0.8, 1.0],
        #'max_depth': [3, 4, 5],
        #'learning_rate': [.01,0.1,0.3],
        #'n_estimators': [100,250,500]}
#clf = GridSearchCV(XGBRegressor(objective= 'reg:squarederror' , random_state=19), pram, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')
#clf.fit(X,Y)

In [16]:
#clf.best_score_, clf.best_params_

In [17]:
def rf_model(x):

    test_1 = df2[df2.EXP == x]
    train_1 = df2[df2.EXP != x]

    train_1 = train_1[['Date','Type', 'from',
           'max_wind_speed', 'R1', 'R2',
           'Pressure_reduced_to_MSL_.Pa..0.MSL',
           'Derived_radar_reflectivity_.dB..1.HYBL',
           'u.component_of_wind_.m.s..85000.ISBL',
           'v.component_of_wind_.m.s..85000.ISBL', 'wind_speed_85000',
           'wind_shear_85000', 'Wind_speed_.gust._.m.s..0.SFC', 'HYBL_one',
           'HYBL_two', 'HYBL_three', 'HYBL_four', 'HYBL_five', 'diff_temp',
           'geo_cbl', 'veg_sfc', 'best_4_layer', 'ws_700']]


    test_1 = test_1[['Date','Type', 'from',
           'max_wind_speed', 'R1', 'R2',
           'Pressure_reduced_to_MSL_.Pa..0.MSL',
           'Derived_radar_reflectivity_.dB..1.HYBL',
           'u.component_of_wind_.m.s..85000.ISBL',
           'v.component_of_wind_.m.s..85000.ISBL', 'wind_speed_85000',
           'wind_shear_85000', 'Wind_speed_.gust._.m.s..0.SFC', 'HYBL_one',
           'HYBL_two', 'HYBL_three', 'HYBL_four', 'HYBL_five', 'diff_temp',
           'geo_cbl', 'veg_sfc', 'best_4_layer', 'ws_700']]

    x_train = train_1.drop('max_wind_speed', 1)
    x_train.drop(['Date','Type'], axis=1, inplace=True)
    y_train = train_1.max_wind_speed
    x_test = test_1.drop('max_wind_speed', 1)
    x_test_copy = x_test.copy()
    x_test.drop(['Date','Type'], axis=1, inplace=True)
    y_test = test_1.max_wind_speed


    model = XGBRegressor(booster='gbtree', objective='reg:squarederror', colsample_bytree = 0.6,
                        learning_rate = 0.1, max_depth=5, n_estimators=500, subsample = 0.6,
                         random_state=16,n_jobs=-1)
    model.fit(x_train, y_train)

    pred_train = model.predict(x_train)
    train_info = model_info2(y_train, pred_train, x , "train")

    y_train30 = np.where(y_train > 30, 1, 0 )
    pred_train30 = np.where(pred_train > 30, 1, 0 )
    train_info2 = model_info1(y_train30, pred_train30, x,'train')


    pred_test = model.predict(x_test)
    test_info = model_info2(y_test, pred_test, x , "test")

    y_test30 = np.where(y_test > 30, 1, 0 )
    pred_test30 = np.where(pred_test > 30, 1, 0 )
    test_info2 = model_info1(y_test30, pred_test30, x,'test')

    table = train_info.append(test_info)
    table2 = train_info2.append(test_info2)
    
    print(x)
    print(table)
    print("==" *10)
    print(table2)

    df4 = pd.DataFrame(y_test)
    df4 = df4.rename(columns={'max_wind_speed': 'actual'})
    df4 = df4.reset_index()
    df4.drop('index', axis=1, inplace=True)
    x_test_copy = x_test_copy.reset_index()
    x_test_copy.drop("index", axis=1, inplace=True)
    df5 = pd.concat([x_test_copy, df4], axis=1)
    pred_df = pd.DataFrame(pred_test, columns=["pred"])
    df6 = pd.concat([df5, pred_df], axis=1)
    df6['From'] = np.where(df6['from'] == 1, 'Mesonet', 'ASOS')
    df6.drop('from', inplace=True, axis=1)
    df6['test_date'] = x
    return(table, table2, df6)

In [18]:
table = pd.DataFrame()
table_class = pd.DataFrame()
data = pd.DataFrame()
for i in df['EXP'].unique():
    t,tc,d = rf_model(i)
    table = table.append(t)
    table_class = table_class.append(tc)
    data = data.append(d)

1/7/17
        MAE        MSE  Residual Mean test_date data_tested
0  2.467257  10.516900         0.0002    1/7/17       train
0  2.971370  13.926024         1.5958    1/7/17        test
   Accuracy  Hits  False Alarms  Misses  No Events       CSI  Hit Rate  \
0  0.973504  2189           432    1441      66628  0.538897   0.60303   
0  1.000000     0             0       0       1959  0.000000   0.00000   

   False Alarm Rate test_date data_tested greter_than_30_knots  \
0          0.164823    1/7/17       train                 3630   
0          0.000000    1/7/17        test                    0   

   less_than_30_knots  
0               67060  
0                1959  
2/9/17
        MAE        MSE  Residual Mean test_date data_tested
0  2.444517  10.423489         0.0007    2/9/17       train
0  3.653666  21.017256        -1.0929    2/9/17        test
   Accuracy  Hits  False Alarms  Misses  No Events       CSI  Hit Rate  \
0  0.973707  2118           397    1459      66614  0.5329

1/4/18
        MAE        MSE  Residual Mean test_date data_tested
0  2.470077  10.557065        -0.0051    1/4/18       train
0  4.092643  30.543353         1.7808    1/4/18        test
   Accuracy  Hits  False Alarms  Misses  No Events       CSI  Hit Rate  \
0  0.974548  1982           381    1405      66404  0.526008  0.585179   
0  0.913201    29             1     214       2233  0.118852  0.119342   

   False Alarm Rate test_date data_tested  greter_than_30_knots  \
0          0.161236    1/4/18       train                  3387   
0          0.033333    1/4/18        test                   243   

   less_than_30_knots  
0               66785  
0                2234  
3/2/18
        MAE        MSE  Residual Mean test_date data_tested
0  2.427127  10.221432         0.0010    3/2/18       train
0  4.728076  37.583447        -0.8946    3/2/18        test
   Accuracy  Hits  False Alarms  Misses  No Events       CSI  Hit Rate  \
0  0.974337  2049           398    1404      66368  0.5

9/10/18
        MAE        MSE  Residual Mean test_date data_tested
0  2.452463  10.439702         0.0002   9/10/18       train
0  3.561545  22.232356         1.7692   9/10/18        test
   Accuracy  Hits  False Alarms  Misses  No Events       CSI  Hit Rate  \
0  0.973404  2171           418    1454      66344  0.536977  0.598897   
0  0.997790     0             0       5       2257  0.000000  0.000000   

   False Alarm Rate test_date data_tested  greter_than_30_knots  \
0          0.161452   9/10/18       train                  3625   
0               NaN   9/10/18        test                     5   

   less_than_30_knots  
0               66762  
0                2257  
9/26/18
        MAE        MSE  Residual Mean test_date data_tested
0  2.432577  10.252488         0.0042   9/26/18       train
0  3.859869  23.814683         1.9681   9/26/18        test
   Accuracy  Hits  False Alarms  Misses  No Events       CSI  Hit Rate  \
0  0.973491  2171           410    1459      66464  0

In [20]:
#table.to_csv('XGB_stats.csv')
table.head()

,MAE,MSE,Residual Mean,test_date,data_tested
0,2.467257,10.516900,0.0002,1/7/17,train
0,2.971370,13.926024,1.5958,1/7/17,test
0,2.444517,10.423489,0.0007,2/9/17,train
0,3.653666,21.017256,-1.0929,2/9/17,test
0,2.417145,10.132724,-0.0034,2/13/17,train


In [22]:
table_class.to_csv('XGB_stats_class.csv')
table_class.head()

,Accuracy,Hits,False Alarms,Misses,No Events,CSI,Hit Rate,False Alarm Rate,test_date,data_tested,greter_than_30_knots,less_than_30_knots
0,0.973504,2189,432,1441,66628,0.538897,0.603030,0.164823,1/7/17,train,3630,67060
0,1.000000,0,0,0,1959,0.000000,0.000000,0.000000,1/7/17,test,0,1959
0,0.973707,2118,397,1459,66614,0.532964,0.592116,0.157853,2/9/17,train,3577,67011
0,0.965551,23,41,30,1967,0.244681,0.433962,0.640625,2/9/17,test,53,2008
0,0.976391,1746,326,1343,67277,0.511274,0.565231,0.157336,2/13/17,train,3089,67603


### 10 Samples 

In [8]:
def rf_model2(x,sample):

    test_1 = df2[df2.EXP == x]
    train_1 = df2[df2.EXP != x]
    exp = train_1['EXP'].unique()
    smp = np.random.choice(exp,sample,replace=False)
    train2 = train_1[train_1['EXP'].isin(smp)]
    print(len(train2['EXP'].unique()))

    train2 = train2[['Type', 'from',
       'max_wind_speed', 'R1', 'R2',
       'Pressure_reduced_to_MSL_.Pa..0.MSL',
       'Derived_radar_reflectivity_.dB..1.HYBL',
       'u.component_of_wind_.m.s..85000.ISBL',
       'v.component_of_wind_.m.s..85000.ISBL', 'wind_speed_85000',
       'wind_shear_85000', 'Wind_speed_.gust._.m.s..0.SFC', 'HYBL_one',
       'HYBL_two', 'HYBL_three', 'HYBL_four', 'HYBL_five', 'diff_temp',
       'geo_cbl', 'veg_sfc', 'best_4_layer', 'ws_700']]


    test_1 = test_1[['Type', 'from',
       'max_wind_speed', 'R1', 'R2',
       'Pressure_reduced_to_MSL_.Pa..0.MSL',
       'Derived_radar_reflectivity_.dB..1.HYBL',
       'u.component_of_wind_.m.s..85000.ISBL',
       'v.component_of_wind_.m.s..85000.ISBL', 'wind_speed_85000',
       'wind_shear_85000', 'Wind_speed_.gust._.m.s..0.SFC', 'HYBL_one',
       'HYBL_two', 'HYBL_three', 'HYBL_four', 'HYBL_five', 'diff_temp',
       'geo_cbl', 'veg_sfc', 'best_4_layer', 'ws_700']]

    x_train = train2.drop('max_wind_speed', 1)
    x_train.drop('Type', axis=1, inplace=True)
    y_train = train2.max_wind_speed
    x_test = test_1.drop('max_wind_speed', 1)
    x_test_copy = x_test.copy()
    x_test.drop('Type', axis=1, inplace=True)
    y_test = test_1.max_wind_speed
    
    ll = list(smp)
    dates = " ".join(str(x) for x in ll)

    model = XGBRegressor(booster='gbtree', objective='reg:squarederror', colsample_bytree = 0.6,
                        learning_rate = 0.1, max_depth=5, n_estimators=500, subsample = 0.6,
                         random_state=16,n_jobs=-1)
    model.fit(x_train, y_train)

    pred_train = model.predict(x_train)
    train_info = model_info(y_train, pred_train, x , "train",)

    pred_test = model.predict(x_test)
    test_info = model_info(y_test, pred_test, x , "test",)

    table = train_info.append(test_info)
    table['train_dates'] = dates
    table['number_sample'] = sample
    print(x)
    print(table)
    print("--" * 40)

    df4 = pd.DataFrame(y_test)
    df4 = df4.rename(columns={'max_wind_speed': 'actual'})
    df4 = df4.reset_index()
    df4.drop('index', axis=1, inplace=True)
    x_test_copy = x_test_copy.reset_index()
    x_test_copy.drop("index", axis=1, inplace=True)
    df5 = pd.concat([x_test_copy, df4], axis=1)
    pred_df = pd.DataFrame(pred_test, columns=["pred"])
    df6 = pd.concat([df5, pred_df], axis=1)
    df6['From'] = np.where(df6['from'] == 1, 'Mesonet', 'ASOS')
    df6.drop('from', inplace=True, axis=1)
    df6['test_date'] = x
    df6['number_sample'] = sample
    return (table, df6)

In [9]:
table = pd.DataFrame()
data = pd.DataFrame()
count = 0
for i in df['EXP'].unique():
    t,d = rf_model2(i,10)
    table = table.append(t)
    data = data.append(d)
    count += 1
    print(count,'/32')

10


/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


1/7/17
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  1.713781   5.127535  0.918707         0.0029    1/7/17       train   
0  3.146377  16.660902  0.435652         0.0467    1/7/17        test   

                                         train_dates  number_sample  
0  6/19/17 10/2/18 5/26/18 12/9/17 3/22/17 9/10/1...             10  
0  6/19/17 10/2/18 5/26/18 12/9/17 3/22/17 9/10/1...             10  
--------------------------------------------------------------------------------
1 /32
10


/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


2/9/17
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  1.999566   6.747597  0.915365        -0.0049    2/9/17       train   
0  3.703234  22.168337  0.530080        -0.0786    2/9/17        test   

                                         train_dates  number_sample  
0  10/12/18 6/13/18 2/24/19 12/17/18 9/10/18 7/13...             10  
0  10/12/18 6/13/18 2/24/19 12/17/18 9/10/18 7/13...             10  
--------------------------------------------------------------------------------
2 /32
10


...

12/17/18
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  1.885076   6.092700  0.924356        -0.0005  12/17/18       train   
0  5.185347  45.084056  0.274486         1.9284  12/17/18        test   

                                         train_dates  number_sample  
0  10/27/18 10/12/18 3/22/17 8/17/18 10/2/18 7/17...             10  
0  10/27/18 10/12/18 3/22/17 8/17/18 10/2/18 7/17...             10  
--------------------------------------------------------------------------------
30 /32
10


/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


12/22/18
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  1.795957   5.467937  0.932406        -0.0029  12/22/18       train   
0  5.617977  49.378404  0.100235        -1.9234  12/22/18        test   

                                         train_dates  number_sample  
0  3/21/18 10/27/18 3/14/17 9/6/18 1/4/18 3/2/17 ...             10  
0  3/21/18 10/27/18 3/14/17 9/6/18 1/4/18 3/2/17 ...             10  
--------------------------------------------------------------------------------
31 /32
10


/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


2/24/19
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  1.816909   5.662513  0.930226        -0.0016   2/24/19       train   
0  5.761192  57.271697  0.422972         0.8290   2/24/19        test   

                                         train_dates  number_sample  
0  10/29/17 9/26/18 12/9/17 3/14/17 6/19/17 2/13/...             10  
0  10/29/17 9/26/18 12/9/17 3/14/17 6/19/17 2/13/...             10  
--------------------------------------------------------------------------------
32 /32


In [10]:
#table.to_csv('10_sample_XGB_stats.csv')
#data.to_csv('10_sample_XGB_data.csv')

In [11]:
table.groupby('data_tested').agg({'MAE':'mean', 'R2':'mean', 'Residual Mean':'mean'})

,MAE,R2,Residual Mean
data_tested,,,
test,4.720789,0.067072,0.065509
train,1.770843,0.921747,-0.000369


In [12]:
res = data['actual'] - data['pred']
res.mean()

-0.025091928456275867

### 15 Samples

In [13]:
table = pd.DataFrame()
data = pd.DataFrame()
count = 0
for i in df['EXP'].unique():
    t,d = rf_model2(i,15)
    table = table.append(t)
    data = data.append(d)
    count += 1
    print(count,'/32')

15


/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


1/7/17
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  2.009856   6.900102  0.892384         0.0025    1/7/17       train   
0  2.784048  12.291409  0.583658         0.8713    1/7/17        test   

                                         train_dates  number_sample  
0  9/26/18 9/4/18 9/6/18 11/19/17 10/12/18 7/13/1...             15  
0  9/26/18 9/4/18 9/6/18 11/19/17 10/12/18 7/13/1...             15  
--------------------------------------------------------------------------------
1 /32
15


/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


2/9/17
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  2.054708   7.189559  0.911929        -0.0049    2/9/17       train   
0  3.705897  21.571960  0.542722        -0.2446    2/9/17        test   

                                         train_dates  number_sample  
0  9/10/18 10/29/17 5/26/18 7/21/18 9/26/18 10/12...             15  
0  9/10/18 10/29/17 5/26/18 7/21/18 9/26/18 10/12...             15  
--------------------------------------------------------------------------------
2 /32
15


...

12/17/18
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  2.133942   7.720601  0.893356        -0.0019  12/17/18       train   
0  4.598832  35.996113  0.420734         1.7425  12/17/18        test   

                                         train_dates  number_sample  
0  12/22/18 7/21/18 2/24/19 9/26/18 9/10/18 10/12...             15  
0  12/22/18 7/21/18 2/24/19 9/26/18 9/10/18 10/12...             15  
--------------------------------------------------------------------------------
30 /32
15


/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


12/22/18
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  1.893636   6.270097  0.892797        -0.0001  12/22/18       train   
0  5.488111  47.532818  0.133865        -0.3397  12/22/18        test   

                                         train_dates  number_sample  
0  2/25/17 3/21/18 6/19/17 7/21/18 1/7/17 5/26/18...             15  
0  2/25/17 3/21/18 6/19/17 7/21/18 1/7/17 5/26/18...             15  
--------------------------------------------------------------------------------
31 /32
15


/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


2/24/19
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  2.078888   7.342101  0.889274         0.0018   2/24/19       train   
0  5.621470  49.246335  0.503830        -1.4916   2/24/19        test   

                                         train_dates  number_sample  
0  9/26/18 10/2/18 1/4/18 3/7/18 2/9/17 2/13/17 6...             15  
0  9/26/18 10/2/18 1/4/18 3/7/18 2/9/17 2/13/17 6...             15  
--------------------------------------------------------------------------------
32 /32


In [14]:
#table.to_csv('15_sample_XGB_stats.csv')
#data.to_csv('15_sample_XGB_data.csv')

In [15]:
table.groupby('data_tested').agg({'MAE':'mean', 'R2':'mean', 'Residual Mean':'mean'})

,MAE,R2,Residual Mean
data_tested,,,
test,4.358969,0.202420,-0.076853
train,2.053423,0.897449,-0.000547


In [16]:
res = data['actual'] - data['pred']
res.mean()

-0.12989475670624498

### 20 Samples

In [17]:
table = pd.DataFrame()
data = pd.DataFrame()
count = 0
for i in df['EXP'].unique():
    t,d = rf_model2(i,20)
    table = table.append(t)
    data = data.append(d)
    count += 1
    print(count,'/32')

20


/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


1/7/17
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  2.209770   8.495803  0.898983        -0.0033    1/7/17       train   
0  3.066536  14.677407  0.502838         1.2692    1/7/17        test   

                                         train_dates  number_sample  
0  9/4/18 9/26/18 3/2/17 1/4/18 12/9/17 10/2/18 2...             20  
0  9/4/18 9/26/18 3/2/17 1/4/18 12/9/17 10/2/18 2...             20  
--------------------------------------------------------------------------------
1 /32
20


/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


2/9/17
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  2.190780   8.282150  0.879614        -0.0028    2/9/17       train   
0  3.458505  19.375227  0.589288         0.2983    2/9/17        test   

                                         train_dates  number_sample  
0  10/29/17 12/22/18 9/10/18 3/7/18 9/4/18 6/19/1...             20  
0  10/29/17 12/22/18 9/10/18 3/7/18 9/4/18 6/19/1...             20  
--------------------------------------------------------------------------------
2 /32
20


...

12/17/18
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  2.095600   7.594650  0.895838         0.0002  12/17/18       train   
0  4.534687  33.925307  0.454058         0.8745  12/17/18        test   

                                         train_dates  number_sample  
0  1/4/18 5/26/18 3/14/17 3/21/18 2/13/17 12/22/1...             20  
0  1/4/18 5/26/18 3/14/17 3/21/18 2/13/17 12/22/1...             20  
--------------------------------------------------------------------------------
30 /32
20


/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


12/22/18
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  2.115453   7.786222  0.892574        -0.0004  12/22/18       train   
0  4.620923  32.809627  0.402148         0.6716  12/22/18        test   

                                         train_dates  number_sample  
0  1/7/17 3/14/17 10/12/18 9/4/18 7/17/18 3/7/18 ...             20  
0  1/7/17 3/14/17 10/12/18 9/4/18 7/17/18 3/7/18 ...             20  
--------------------------------------------------------------------------------
31 /32
20


/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


2/24/19
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  2.158136   7.969725  0.895840        -0.0009   2/24/19       train   
0  5.954072  56.316296  0.432598        -2.2944   2/24/19        test   

                                         train_dates  number_sample  
0  2/13/17 3/2/17 6/19/17 3/2/18 9/4/18 3/14/17 6...             20  
0  2/13/17 3/2/17 6/19/17 3/2/18 9/4/18 3/14/17 6...             20  
--------------------------------------------------------------------------------
32 /32


In [18]:
#table.to_csv('20_sample_XGB_stats.csv')
#data.to_csv('20_sample_XGB_data.csv')

In [19]:
table.groupby('data_tested').agg({'MAE':'mean', 'R2':'mean', 'Residual Mean':'mean'})

,MAE,R2,Residual Mean
data_tested,,,
test,4.206500,0.264383,0.110794
train,2.205694,0.885252,0.000041


In [20]:
res = data['actual'] - data['pred']
res.mean()

0.041527097089895174

### Samples 25

In [21]:
table = pd.DataFrame()
data = pd.DataFrame()
count = 0
for i in df['EXP'].unique():
    t,d = rf_model2(i,25)
    table = table.append(t)
    data = data.append(d)
    count += 1
    print(count,'/32')

25


/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


1/7/17
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  2.320189   9.363363  0.882486         0.0062    1/7/17       train   
0  2.818212  12.478459  0.577322         1.1294    1/7/17        test   

                                         train_dates  number_sample  
0  3/22/17 9/4/18 6/19/17 5/26/18 5/15/18 7/13/17...             25  
0  3/22/17 9/4/18 6/19/17 5/26/18 5/15/18 7/13/17...             25  
--------------------------------------------------------------------------------
1 /32
25


/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


2/9/17
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  2.196972   8.295900  0.882447         0.0009    2/9/17       train   
0  3.599829  20.434654  0.566830        -1.0828    2/9/17        test   

                                         train_dates  number_sample  
0  10/2/18 9/6/18 3/21/18 3/22/17 9/10/18 9/26/18...             25  
0  10/2/18 9/6/18 3/21/18 3/22/17 9/10/18 9/26/18...             25  
--------------------------------------------------------------------------------
2 /32
25


...

12/17/18
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  2.357504   9.537863  0.856733         0.0001  12/17/18       train   
0  4.570402  34.231102  0.449137         1.1634  12/17/18        test   

                                         train_dates  number_sample  
0  3/14/17 6/19/17 11/19/17 5/15/18 12/22/18 1/7/...             25  
0  3/14/17 6/19/17 11/19/17 5/15/18 12/22/18 1/7/...             25  
--------------------------------------------------------------------------------
30 /32
25


/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


12/22/18
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  2.185330   8.279074  0.875905         0.0025  12/22/18       train   
0  4.415142  30.178561  0.450091         0.5210  12/22/18        test   

                                         train_dates  number_sample  
0  10/27/18 1/4/18 9/10/18 3/7/18 12/9/17 9/4/18 ...             25  
0  10/27/18 1/4/18 9/10/18 3/7/18 12/9/17 9/4/18 ...             25  
--------------------------------------------------------------------------------
31 /32
25


/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


2/24/19
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  2.364502   9.595022  0.867885        -0.0007   2/24/19       train   
0  6.154267  60.792304  0.387501        -2.3719   2/24/19        test   

                                         train_dates  number_sample  
0  9/10/18 8/17/18 3/21/18 3/2/18 9/26/18 7/21/18...             25  
0  9/10/18 8/17/18 3/21/18 3/2/18 9/26/18 7/21/18...             25  
--------------------------------------------------------------------------------
32 /32


In [22]:
#table.to_csv('25_sample_XGB_stats.csv')
#data.to_csv('25_sample_XGB_data.csv')

In [23]:
table.groupby('data_tested').agg({'MAE':'mean', 'R2':'mean', 'Residual Mean':'mean'})

,MAE,R2,Residual Mean
data_tested,,,
test,4.164603,0.290361,0.075344
train,2.320303,0.873993,0.001275


In [24]:
res = data['actual'] - data['pred']
res.mean()

0.022984543286979436

### 30 Samples 

In [25]:
table = pd.DataFrame()
data = pd.DataFrame()
count = 0
for i in df['EXP'].unique():
    t,d = rf_model2(i,30)
    table = table.append(t)
    data = data.append(d)
    count += 1
    print(count,'/32')

30


/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


1/7/17
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  2.457562  10.469138  0.860387         0.0038    1/7/17       train   
0  2.844239  12.728281  0.568860         1.1870    1/7/17        test   

                                         train_dates  number_sample  
0  10/2/18 12/22/18 6/13/18 5/15/18 1/4/18 2/9/17...             30  
0  10/2/18 12/22/18 6/13/18 5/15/18 1/4/18 2/9/17...             30  
--------------------------------------------------------------------------------
1 /32
30


/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


2/9/17
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  2.442085  10.306441  0.863906         0.0059    2/9/17       train   
0  3.637721  20.856422  0.557890        -1.1957    2/9/17        test   

                                         train_dates  number_sample  
0  2/13/17 9/10/18 12/17/18 10/2/18 2/25/17 5/26/...             30  
0  2/13/17 9/10/18 12/17/18 10/2/18 2/25/17 5/26/...             30  
--------------------------------------------------------------------------------
2 /32
30


...

12/17/18
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  2.432083  10.260085  0.862073         0.0032  12/17/18       train   
0  4.255255  29.477987  0.525627         0.6585  12/17/18        test   

                                         train_dates  number_sample  
0  10/2/18 10/29/17 3/22/17 2/13/17 2/24/19 3/2/1...             30  
0  10/2/18 10/29/17 3/22/17 2/13/17 2/24/19 3/2/1...             30  
--------------------------------------------------------------------------------
30 /32
30


/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


12/22/18
        MAE       MSE        R2  Residual Mean test_date data_tested  \
0  2.425626  10.15709  0.863526         0.0005  12/22/18       train   
0  4.169786  27.67812  0.495654        -0.2654  12/22/18        test   

                                         train_dates  number_sample  
0  3/7/18 7/21/18 5/15/18 3/2/17 10/29/17 9/26/18...             30  
0  3/7/18 7/21/18 5/15/18 3/2/17 10/29/17 9/26/18...             30  
--------------------------------------------------------------------------------
31 /32
30


/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/envs/coding/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


2/24/19
        MAE        MSE        R2  Residual Mean test_date data_tested  \
0  2.378014   9.803600  0.857731        -0.0024   2/24/19       train   
0  5.840580  53.741277  0.458542        -1.5570   2/24/19        test   

                                         train_dates  number_sample  
0  1/7/17 1/4/18 3/22/17 10/12/18 9/6/18 5/15/18 ...             30  
0  1/7/17 1/4/18 3/22/17 10/12/18 9/6/18 5/15/18 ...             30  
--------------------------------------------------------------------------------
32 /32


In [26]:
#table.to_csv('30_sample_XGB_stats.csv')
#data.to_csv('30_sample_XGB_data.csv')

In [27]:
table.groupby('data_tested').agg({'MAE':'mean', 'R2':'mean', 'Residual Mean':'mean'})

,MAE,R2,Residual Mean
data_tested,,,
test,4.025470,0.338013,-0.062856
train,2.427776,0.861766,0.001378


In [28]:
res = data['actual'] - data['pred']
res.mean()

-0.0997783277667618